In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
# df_sgm = pd.read_csv('df_trans_user_item_date.csv', sep='\t', dtype='object')
# df_item = pd.read_csv('MASTER_ITEM_202012121853.csv', dtype='object')
# df_sgm['TR_DATE'] = df_sgm['TR_DATE'].map(lambda x: eval(x))
# df_sgm['TR_PLU'] = df_sgm['TR_PLU'].apply(lambda x: x[0:-1])

In [3]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [4]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like '%sgm eksplor%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu[0:20]:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_1 = pd.concat(df_all)
df_seq_1.head()

,_id,TR_MEMBERS,TR_PLU_PCS,TR_DATE,updated_at,TRX_DATE,last_order_date,next_order_date
0,5fd90a51b8f8002aaaba12f1,9990013045411848,120768,{'1607904000': 5},2020-12-20 20:31:05.342,{'1607904000': 1},NaT,NaT
1,5fd90a51b8f8002aaaba36a1,9990010296470971,120768,"{'1607904000': 5, '1605571200': 2}",2020-12-21 00:03:15.446,"{'1607904000': 1, '1605571200': 1}",NaT,NaT
2,5fd90a52b8f8002aaaba8f23,9990012980982737,120768,{'1607904000': 5},2020-12-20 20:32:06.915,{'1607904000': 1},NaT,NaT
3,5fd90a53b8f8002aaabab9df,9990012808033100,120768,{'1607904000': 5},2020-12-20 20:32:24.500,{'1607904000': 1},NaT,NaT
4,5fd90a54b8f8002aaabb0e2a,9990012273495228,120768,"{'1607904000': 5, '1607040000': 2, '1606176000': 2, '1605916800': 2, '1605657600': 2, '1605225600': 2, '1602288000': 1, '1608163200': 1}",2020-12-21 06:27:33.763,"{'1608163200': 1, '1607904000': 1, '1607040000': 1, '1606176000': 1, '1605916800': 1, '1605657600': 1, '1605225600': 1, '1602288000': 1}",2020-12-14,2020-12-15


In [ ]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like '%uht%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu[0:20]:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_2 = pd.concat(df_all)
df_seq_2.head()

In [ ]:
df_seq = pd.concat([df_seq_1, df_seq_2]).reset_index(drop=True)

In [ ]:
q_nas = """
SELECT ITEM_N, DESCP, UKURAN, PANJANG, LEBAR, TINGGI FROM MASTER_NAS_M mnm 
"""

df_nas = pd.read_sql_query(q_nas, connection)
df_nas.head()

In [ ]:
df = pd.merge(df_seq, df_nas, left_on='TR_PLU_PCS', right_on='ITEM_N')[list(df_seq) + ['DESCP','UKURAN']]
df = df[df['TRX_DATE'].notnull()]
df_hist = df[df['TRX_DATE'].apply(lambda x: len(x)) >= 6]

In [ ]:
def cal_feature(set_timestamp):

    set_timestamp = sorted(set_timestamp.items())
    last_order = int(set_timestamp[-1][0])
    
    li_seq = []
    for idx in range(0, len(set_timestamp)-1, 1):

        diff = int(set_timestamp[idx+1][0]) - int(set_timestamp[idx][0])
        diff_day = int((diff / 3600) / 24)
        li_seq.append(diff_day)
        
    

    li_seq = li_seq[-5:]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)


    return li_seq, last_order

In [ ]:
df_hist['tr_date_seq'], df_hist['last_ord1er'] = zip(*df_hist['TRX_DATE'].apply(cal_feature))

In [ ]:
col_feature = ['t_5', 't_4', 't_3', 't_2', 't_1', 'max', 'min', 'since', 'mean', 'variance']
train = pd.DataFrame(df_hist.tr_date_seq.tolist(), index=df_hist.index, columns=col_feature)

# X = train.drop('t_0', 1)
# Y = train['t_0']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

### Compare time consuming in different netto
> 
* first we user with less variance, we assume user buy based on their consumption and less affected by promotion or random purchase date
* age might be affect consumption, cause more mature, amount of consumption should be decreased by parent

In [ ]:
# with open('watchlist_item.txt') as f:
#     watchlist = f.read().splitlines()
#     df_watchlist = pd.DataFrame({'DESCP':watchlist})

In [ ]:
train_descp = pd.concat([train,df_hist], 1)[col_feature + ['TR_MEMBERS', 'DESCP', 'TRX_DATE', 'UKURAN']]
# train_descp = pd.merge(df_watchlist, train_descp, left_on='DESCP', right_on='DESCP', how='left')
train_descp = train_descp[train_descp['UKURAN'].notnull()]
train_descp['UKURAN'] = train_descp['UKURAN'].astype(int)

In [ ]:
# train_descp['DESCP_NET'] = train_descp['DESCP'].str.split(' ').str[-1]

In [ ]:
train_descp.groupby('UKURAN').agg({'mean':'mean'})

In [ ]:
train_descp_cond = train_descp[train_descp['variance'] < 100]
# display(train_descp_cond.head(3))
train_descp_cond.groupby('UKURAN').agg({'mean':'mean'})

In [ ]:
train_descp['DESCP_TYPE'] = train_descp['DESCP'].str.split(' ').str[0:3].str.join(' ')

In [ ]:
train_descp.groupby(['DESCP_TYPE','UKURAN']).agg({'mean':'mean'})

In [ ]:
train_descp_cond = train_descp[train_descp['variance'] < 80]
# display(train_descp_cond.head(3))
train_descp_cond.groupby(['DESCP_TYPE','UKURAN']).agg({'mean':'mean'})